In [236]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [237]:
ALE_COLOR = 'rgba(30,144,255,1)'
PD_COLOR = 'rgba(220,20,60,1)'
ALE_COLOR_LIGHT = 'rgba(30,144,255,0.2)'
PD_COLOR_LIGHT = 'rgba(220,20,60,0.2)'

In [238]:
CSV_PATH = "results_final/all_rows.csv"
METRIC_NAMES = {
    "rho": "Spearman rho",
    "l1": "L1",
    "l2": "L2",
    "max_diff": "maximum distance (Linf)"    
}

In [239]:
df = pd.read_csv(CSV_PATH).drop(columns=['Unnamed: 0']).drop_duplicates()
df.head()

,name,path,variable,size,seed,lr,iter,constrain,dist_weight,ale_l2,ale_l1,ale_max_diff,ale_rho,pd_l2,pd_l1,pd_max_diff,pd_rho
0,heart,results_final/heart/age_512_0_gradient_0.1_50_...,age,512,0,0.1,50,False,0.01,0.025600,0.025600,0.067292,-0.859558,0.038559,0.038559,0.078188,-0.994805
1,heart,results_final/heart/age_256_0_gradient_0.1_50_...,age,256,0,0.1,50,False,0.01,0.020159,0.020159,0.043379,-0.842653,0.041103,0.041103,0.104379,-0.885714
2,heart,results_final/heart/age_128_0_gradient_0.1_50_...,age,128,0,0.1,50,False,0.01,0.018468,0.018468,0.044442,-0.434330,0.025585,0.025585,0.052349,-0.984416
3,heart,results_final/heart/age_64_0_gradient_0.1_50_0...,age,64,0,0.1,50,False,0.01,0.011359,0.011359,0.023311,-0.304291,0.011301,0.011301,0.028792,-0.976623
5,heart,results_final/heart/age_32_0_gradient_0.1_50_0...,age,32,0,0.1,50,False,0.01,0.024567,0.024567,0.087358,-0.890767,0.033288,0.033288,0.084878,-0.942857


In [240]:
df = df[df["size"].isin([32, 64, 128])]
df = df[df.dist_weight.isin([0, 0.0001, 0.001, 0.01, 0.1, 1])]

In [241]:
def split_rows(df):    
    orig_len = len(df)
    for metric in ("l1", "l2", "max_diff", "rho"):
        df[metric] = df[[f"ale_{metric}", f"pd_{metric}"]].values.tolist()
        df = df.explode(metric)

    cond_ale = (df.l1 == df.ale_l1) & (df.l2 == df.ale_l2) & (df.ale_max_diff == df.max_diff) & (df.ale_rho == df.rho) 
    cond_pd = (df.l1 == df.pd_l1) & (df.l2 == df.pd_l2) & (df.pd_max_diff == df.max_diff) & (df.pd_rho == df.rho) 

    df = df[cond_ale | cond_pd]
    # assert len(df) == 2 * orig_len, f"{len(df)}, {2 *orig_len}"

    cond_ale = (df.l1 == df.ale_l1) & (df.l2 == df.ale_l2) & (df.ale_max_diff == df.max_diff) & (df.ale_rho == df.rho) 
    cond_pd = (df.l1 == df.pd_l1) & (df.l2 == df.pd_l2) & (df.pd_max_diff == df.max_diff) & (df.pd_rho == df.rho)
    df.loc[cond_ale, "explanation"] = "ale"
    df.loc[cond_pd, "explanation"] = "pd"

    return df

In [242]:
df = split_rows(df)

In [243]:
def boxplots(variable, metric="rho", df=df):
    df["split variable"] = f"{variable} " + df[variable].astype(str)
    df = df.sort_values(by=["name", "variable", variable])
    groups = df.groupby(["name", "variable"])
    for (n, v), group in groups:
        group.variable = group.variable.astype(str)
        fig = px.box(group, x="split variable", y=metric, color="explanation", title=f"{n}, variable {v}, split by {variable}")
        fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
        fig.show()

In [244]:
boxplots("size")

In [245]:
boxplots("dist_weight")

In [246]:
boxplots("dist_weight", "l2")

In [247]:
boxplots("size", "l2")